In [2]:
import json

# Load all three datasets
with open("data/youtube.json") as f:
    yt = json.load(f)
with open("data/podcasts.json") as f:
    podcasts = json.load(f)
with open("data/webpages_by_tag_rich.json") as f:
    pages = json.load(f)

# Normalize all into one consistent structure
contents = []
for c in yt + podcasts + pages:
    contents.append({
        "title": c.get("title"),
        "description": c.get("description", ""),
        "source": c.get("source", "youtube" if "youtube" in c.get("link","") else "unknown"),
        "url": c.get("link") or c.get("url"),
        "tags": c.get("tags") if "tags" in c else [c.get("tag")] if "tag" in c else []
    })

In [4]:
pip install -q sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [5]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('all-MiniLM-L6-v2')

# Load tag names from your tags.json
with open("data/tags.json") as f:
    tags_data = json.load(f)

all_tags = []
for cat in tags_data["categories"].values():
    all_tags.extend(cat["tags"])

# Embed all tag strings once
tag_embeds = model.encode(all_tags, convert_to_tensor=True)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
tagged_contents = []

for c in contents:
    # Combine title + description for embedding
    text = (c.get("title", "") or "") + " " + (c.get("description", "") or "")
    content_embed = model.encode(text, convert_to_tensor=True)
    scores = util.cos_sim(content_embed, tag_embeds)[0]
    
    # Get top 3 matching tags
    top_tags_idx = scores.topk(3).indices
    top_tags = [all_tags[i] for i in top_tags_idx]

    # Merge existing and new tags (avoid duplicates)
    existing_tags = c.get("tags", [])
    combined_tags = list(dict.fromkeys(existing_tags + top_tags))  # preserves order & removes duplicates

    # Update and store
    c["tags"] = combined_tags
    tagged_contents.append(c)

In [8]:
for c in tagged_contents:
    print(f"🎬 {c['title']}")
    print("   → Tags:", ", ".join(c["tags"]))
    print()

🎬 What is AI Ethics?
   → Tags: AI ethics, Artificial Intelligence, Data Science, Economics

🎬 AI Ethics | Ethics Defined
   → Tags: AI ethics, Artificial Intelligence, Psychology, Economics

🎬 The Ethics of AI│Stuart J. Russell (University of California, Berkeley, Professor)
   → Tags: AI ethics, Artificial Intelligence, Article, Economics

🎬 Ethics & AI: Equal Access and Algorithmic Bias
   → Tags: AI ethics, Artificial Intelligence, Economics, Psychology

🎬 Ethics of AI: Challenges and Governance
   → Tags: AI ethics, Artificial Intelligence, Innovation, Leadership

🎬 AI is an Ethical Nightmare
   → Tags: AI ethics, Artificial Intelligence, Economics, Minimalism

🎬 AI Is Dangerous, but Not for the Reasons You Think | Sasha Luccioni | TED
   → Tags: AI ethics, Artificial Intelligence, Documentaries, True Crime

🎬 How to implement AI Ethics
   → Tags: AI ethics, Artificial Intelligence, Machine Learning, Management

🎬 EU AI Act with Punit Bhatia in the FIT4PRIVACY Podcast E150 S06
   